# Simple plugin development

This chapter covers plugin development where the plugin performs a task on behalf of an operator.

Chapter 3 will cover plugin development where the PCP plugin acts as an intermediary between PCP and another controller (or set of controllers) to perform a function.  The concepts and requirements are exactly the same, but function arguments change

# Framework Defined Environment Variables 


The PCP Environment will prepare environment variables on the plugin's behalf.

1. The plugin must respect the environment 
2. The system shall select at most one integer port (in environment as ASCII string) available for the plugin in the **PORT** key
   * The plugin may bind to that port at any point during it's lifetime
   * The plugin should not attemt to bind/listen to ports other than the system-selected port (user may *hardcode* this port in a seperate subsytem)
3. The system shall provide the hostname/IP of the brain in the **RETHINK_HOST** key
   * The plugin shall use the ControllerPlugin class functions where possible
   * The plugin may use the ramrodbrain (import brain) python package to communicate with the brain directly
4. The system shall provide an indication of release version to the plugin in the **STAGE** key
   * The system shall provide the exactly one ASCII string (one of "DEV" "TESTING", "QA", "PROD")
   * The plugin may use this key as an indication of verbose logging, feature testing, etc
5. The system shall populate the plugin name in the **PLUGIN_NAME** key.
5. The plugin should not modify the enviornment variables.  (Changes will not be observed or respected by the PCP system).  


In [1]:


#This Cell emulates the ramrodpcp system preparing the environment for the plugin
from os import environ
environ["PORT"] = "10000" 
environ["RETHINK_HOST"] = "10.0.0.1"
environ["STAGE"] = "PROD" 
environ['PLUGIN_NAME'] = "SamplePlugin"

# Extend the base class

Every plugin should seek to extend the ControllerPlugin base class.

This guide includes a vendored copy of the base class, but development should acquire the current master branch controller_plugin.py

This project does not guarantee forward or backward API compatibility across major versions

In [2]:
from src.controller_plugin import ControllerPlugin


Imported controller_plugin.py file is vendored from 
   https://github.com/ramrod-project/backend-interpreter/blob/master/src/controller_plugin.py  

Ongoing development should obtain a copy from the ramrod-project/backend-interpreter



# [REQ-C] Author a compatible (but failing) plugin

3 requirements for a compatible plugin name

1. **Naming** The Filename must match the Class name
  1. Filename is SamplePlugin.py, ClassName is SamplePlugin
2. **Advertisemnt** The plugin advertises its command functionality in the  \__init\__ function with a static **list** of commands
  1. The Commands should be static, updating plugin command list during execution is possible but unsupported.
  2. Command syntax is defined in *The Brain Documentation* and can be verified with the brain helper functions
3. **\_start function** Implement a \_start function
  1. the \_ start function should hold control flow for the duration of the plugin's existance.
  2. the plugin may retuen any type (including None) from \_start to instruct the system that the plugin has exited. 

In [6]:
from src.controller_plugin import ControllerPlugin
FUNCTIONALITY = [] # [REQ-C]-2.a
class SamplePlugin(ControllerPlugin): # [REQ-C]-1
    def __init__(self):
        self.name = "SamplePlugin"
        super().__init__(self.name, FUNCTIONALITY) # [REQ-C]-2.b
    def _start(self): # [REQ-C]-3
        return None # [REQ-C]-3.b
plugin = SamplePlugin()

## Note about compatible plugin
The above plugin contains no command functionality and would exit immediately on start via return None statement

This is considered failing [REQ-C]-3 because it is not holding program control flow





## Assumptions about SamplePlugin
* SamplePlugin is a web scraper
* PCP sytem may deploy one or many web scrapers
* Each web scraper asks for a Job containing a URL to scrape
* The web scraper returns the HTML for one URL (without crawling on it's own)



#  [REQ-F] Author a fully functional SamplePlugin
3 requremnts for a fully functional plugin


1. implements a control loop with logic to hold control for the duration of existance
2. Plugin may request a job at any time
  1. Job will be a command mapped to a target (see *The Brain Documentation* for more details)
  2. The Job may be NoneType if there is no Job at the time of request.  
    1. It is the plugin's responsibility to back off job requests (if desired) to conserve resources.
  3. The JobCommand portion of the Job will be the exact command the plugin advertised (filled out by a user)
      1. With the exception of a modified Value parameter (think plug and chug)
      2. in this case job["JobCommand"]["Inputs"][i]["Value"] will be modified
 
3. The Plugin must supply string (or utf-8 encoded bytes) output 
  1. The plugin may supply a blank string if no output is required 


notes: 
* PCP system may kill the process at any time
* The Plugin may request jobs and return output at any time so long as it appropriately tracks Job['id']
* The processing may be in serial or in parallel (this example is serial)

In [9]:
from src.controller_plugin import ControllerPlugin
import requests
cmdWget = {"CommandName" : "wget",
          "Tooltip" : "Retrieves raw webpage content",
          "Output" : True,
          "Inputs" : [{"Name" : "Target IP Address",
                     "Type" : "textbox",
                     "Tooltip" : "Modify the URL to the desired target IP address",
                     "Value" : ""}]}
FUNCTIONALITY = [cmdWget] # [REQ-C]-2.a
class SamplePlugin(ControllerPlugin): # [REQ-C]-1
    def __init__(self):
        self.name = "SamplePlugin"
        super().__init__(self.name, FUNCTIONALITY) # [REQ-C]-2.b
    def _start(self): # [REQ-C]-3
        while True: #  [REQ-F]-1 (simple control loop)
            job = self.request_job() # [REQ-F]-2
            if job: # [REQ-F]-2.B
                url = job['JobCommand']['Inputs'][0]["Value"]
                resposne = requests.get(url)
                if response:
                    output = response.content # preparing for [REQ-F]-3
                else:
                    output = "" # preparing for [REQ-F]-3.A
                self.respond_output(job, output) # [REQ-F]-3

plugin = SamplePlugin()

## Additional Notes

In this sample, the PCP system may invoke dozens or hundreds of SamplePlugins.  The PCP system may provide discrete work to each job.  The plugins should only invoke code to execute the job(s) it's currently operating on.  

### PCP Guarantees
1. The PCP system may spin up a fleet of multiple of the same plugins.  
2. The request_job function will give a job to at most one of the plugins which requests one.
3. The request_job function will deliver the jobs in ascending order of job['StartTime'] 
4. The request_job function will never deliver a job to a plugin where job['ExpireTime'] is in the past.